In [550]:
import pandas as pd
import numpy as np
from datetime import datetime 
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
import os
import sys 
import shutil

import cx_Oracle
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

import sqlite3

# Parameter variable

In [551]:
csv_temp_folder='csv_temp'
csv_error_folder='csv_error'

source_name=''
source_name="yip_invoice_monthly" # df

#source_name="yip_ar_receipt"  # df/csv
#source_name='yip_ap_payment' # csv

#source_name="yip_pj_status" # csv

#source_name='yip_po_listing' # df

#source_name='yip_gl_account' # df

# set True whatever , you want to reload all items
isLoadingAllItems=True
is_py=False

init_date_query='2020-01-01 00:00:00'
# init_date_query='2023-01-01 00:00:00'

data_base_file=r'D:\ETL_Orable_To_BQ\etl_web_admin\etl_config_transaction.db'
json_credential_file=r'C:\Windows\mismgntdata-bigquery--bq-loader-34713c332847.json'

In [552]:
# Enter parameter on script

In [553]:
if is_py:
    press_Y=''
    ok=False

    if len(sys.argv) > 1:
        source_name=sys.argv[1]

        if sys.argv[2]=='0':
         isLoadingAllItems=False
        elif sys.argv[2]=='1'  :
         isLoadingAllItems=True
        else:
            raise Exception("isLoadingAllItems 1=True | 0=False")

        ok=True 

    else:
        print("Enter the following input: ")
        source_name = input("View Table Name : ")
        source_name=source_name.lower()
        load_option= int(input("Loading All Data option (1=True | 0=False): "))
        if load_option==0:
         isLoadingAllItems=False
        elif load_option==1  :
         isLoadingAllItems=True
        else:
            raise Exception("isLoadingAllItems 1=True | 0=False")  

        print(f"Confirm to Load view = {source_name} and Load All Data= {isLoadingAllItems}")
        press_Y=input(f"Press Y=True But any key=False : ") 
        if press_Y=='Y':
         ok=True

    if ok==False:
        print("No any action")
        quit()


# Check temp and error folder

In [554]:
if os.path.exists(csv_temp_folder)==False:
  os.mkdir(csv_temp_folder)
if os.path.exists(csv_error_folder)==False:
  os.mkdir(csv_error_folder)  

# Init Variable

In [555]:
listError=[]

projectId='mismgntdata-bigquery'
region='asia-southeast1'
dataset_id='MIS_BI_DW'
table_id = f"{projectId}.{dataset_id}.{source_name}"

_ip='172.30.57.10' #'172.30.57.10'
_hostname='YIPGERP'
_port=1521
_servicename='PROD'
_username='yipgbi'
_password='yipgbi'

host = 'mail.yipintsoi.com'
port=  25
sender="mis-bi-service@yipintsoigroup.com"
receivers=['pongthorn.sa@yipintsoi.com']
#receivers=['pongthorn.sa@yipintsoi.com','mis-bi-service@yipintsoigroup.com']


sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))


temp_path=f'{csv_temp_folder}/{source_name}.csv'


start_date_query=''
updateCol='last_update_date'

# Setting BigQuery and Check DataSet

In [557]:
credentials = service_account.Credentials.from_service_account_file(json_credential_file)
client = bigquery.Client(credentials=credentials, project=projectId)

# dataset
try:
    dataset = client.get_dataset(f"{projectId}.{dataset_id}")
    print("Dataset {} already exists".format(dataset_id))
except Exception as ex:
    msg=f"Dataset {dataset_id} is not found"
    listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg]) 
    logErrorMessage(listError)

Dataset MIS_BI_DW already exists


In [558]:
dt_imported=datetime.now()
dtStr_imported=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
#dtStr_imported='2023-05-23 23:00:00'

dt_imported=datetime.strptime(dtStr_imported,"%Y-%m-%d %H:%M:%S")

print(dtStr_imported)
print(dt_imported)

2023-06-13 16:44:06
2023-06-13 16:44:06


# Email Nofification &  Manage Log Error Message

In [559]:
import  smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def sendMailForError(errorSubject,errorHtml):
    message = MIMEMultipart("alternative")
    message["Subject"] =errorSubject 
    message["From"] = sender
    message["To"] = ','.join(receivers)

    html =errorHtml 

    part_html = MIMEText(html, "html")
    message.attach(part_html)

    try:

        with smtplib.SMTP(host,port) as mail_server:
            #mail_server.login(login, password)
            mail_server.sendmail(sender, receivers, message.as_string())
            print("Successfully sent email")

    except (gaierror, ConnectionRefusedError):
       msg='Failed to connect to the server. Bad connection settings?'
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg]) 
       logErrorMessage(listError)
    except smtplib.SMTPServerDisconnected:
       msg='Failed to connect to the server. Wrong user/password?'
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg]) 
       logErrorMessage(listError)
    except smtplib.SMTPException as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
       logErrorMessage(listError)

    return True
    

In [560]:
def move_error_file(): # if any error ,then move csv to investigte later
    error_csv_path=''
    try:
        if os.path.exists(temp_path):
         error_csv_file=f"{source_name}_error_{ dt_imported.strftime('%d%m%y_%H%M%S')}.csv"   
         new_temp_path=f'{csv_temp_folder}/{error_csv_file}'
         os.rename(temp_path,new_temp_path)  

         error_csv_path=f'{csv_error_folder}/{error_csv_file}'
         shutil.move(new_temp_path,error_csv_path )
         


    except Exception as e:
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
        logErrorMessage(listError)
    return   error_csv_path      
 

In [561]:
def logErrorMessage(errorList,raise_ex=True):
    
    def add_error_to_file(error_des):
        f = open(r'log_error.txt', 'a')
        error_str = f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}|{repr(error_des)}\n'

        f.write(error_str)
        f.close()
        print(error_str)
        raise Exception(error_str)
    
    def add_logError(recordList):
        try:
            sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
            cursor = sqliteConnection.cursor()
            sqlite_insert_query = """
            INSERT INTO log_error
            (error_datetime,etl_datetime, data_source_id,message)  VALUES (?,?,?,?);
             """
            cursor.executemany(sqlite_insert_query, recordList)
            print("Done Log Error")
            sqliteConnection.commit()
            cursor.close()
            
        except Exception as e:
            msg=f"{data_base_file} error : {str(e)}"
            listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
            add_error_to_file(msg)
        finally:
            if sqliteConnection:
                sqliteConnection.close()
            
    if len(errorList)>0:
        
        error_message=f"ETL Error on {source_name} at {dtStr_imported}"
        # move csv error file to examine later
        error_csv_path  =move_error_file()
        if error_csv_path!='':
            error_message= f"{error_csv_path} - {error_message}"
        error_message=f"MIS-BI : {error_message}"   
        
        print(error_message)
        
        dfError=pd.DataFrame(data=errorList,columns=["error_datetime","etl_datetime","data_source_id","message"])
        print(f"Total log error={len(dfError)}")
        print(dfError)
        
        # add to error table into sqlite
        recordError=dfError.to_records(index=False) 
        add_logError(recordError)
        
        # sened main
        emailResult=sendMailForError(error_message, dfError.to_html(index=False))
        
        if raise_ex==True:
         raise  Exception(error_message)
    

# Get & Set Oracle ViewName and other configuration data

In [562]:
# get data from data_source
def get_ds(data_source_name):
   try: 
        conn = sqlite3.connect(os.path.abspath(data_base_file))
        sql_ds=f"""select * from data_source where id='{data_source_name}'  """
        print(sql_ds)
        df_item=pd.read_sql_query(sql_ds, conn)
        if df_item.empty==False:
           return df_item.iloc[0,:]
        else:
           return None
   except Exception as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
       logErrorMessage(listError)
    
ds_item=get_ds(source_name)
ds_item

select * from data_source where id='yip_invoice_monthly'  


id                      yip_invoice_monthly
first_load_col                      gl_date
partition_date_col                  gl_date
partition_date_type                   MONTH
cluster_col_list         gl_date ,dept_name
date_col_list          gl_date,invoice_date
load_from_type                    dataframe
Name: 0, dtype: object

In [563]:
if ds_item is not None:
    print("Load data source config data")

    loading_from=ds_item['load_from_type']
    print(f"Load data into BigQuery from {loading_from}")
    
    colFirstLoad=ds_item['first_load_col']
    colFirstLoad=colFirstLoad.strip().lower()
    print(f"Date Column as condition to load at first = {colFirstLoad}")

    partitionCol=ds_item['partition_date_col']  # required DateTime Type
    partitionCol=partitionCol.strip().lower()
    if   ds_item['partition_date_type']=="DAY":
     partitionType=bigquery.TimePartitioningType.DAY
    elif ds_item['partition_date_type']=="MONTH":
     partitionType=bigquery.TimePartitioningType.MONTH    
    elif ds_item['partition_date_type']=="YEAR":
     partitionType=bigquery.TimePartitioningType.YEAR   
    else:
     partitionType=bigquery.TimePartitioningType.DAY
    
    print(f"Partition : {partitionCol} - {partitionType}")
    
        
    if ds_item['cluster_col_list']=='':
     clusterCols=[]   
     print(f"{clusterCols} (No cluster cols)")   
     
    else:
     clusterCols=  ds_item['cluster_col_list'].split(',') 
     clusterCols = list(map(str.strip,clusterCols))
     clusterCols = list(map(str.lower,clusterCols))   
     print(f"Cluster List: {clusterCols}")

    if ds_item['date_col_list']=='':
     dateCols=[]   
     print(f"{dateCols} (No Date cols)")   
     
    else:
     dateCols=  ds_item['date_col_list'].split(',') 
     dateCols = list(map(str.strip,dateCols))
     dateCols = list(map(str.lower,dateCols))      
     print(f"Date Cols: {dateCols}")

else:
   listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,f"Not found view {source_name} in data_source table"])
   logErrorMessage(listError)

Load data source config data
Load data into BigQuery from dataframe
Date Column as condition to load at first = gl_date
Partition : gl_date - MONTH
Cluster List: ['gl_date', 'dept_name']
Date Cols: ['gl_date', 'invoice_date']


# List Last ETL Transacton by Datasource Name
### Get last etl of the specific view to perform incremental update

In [524]:
def get_last_etl_by_ds(data_source):
   try: 
    conn = sqlite3.connect(os.path.abspath(data_base_file))
    sql_last_etl=f"""select etl_datetime,data_source_id from etl_transaction where data_source_id='{data_source}' 
    order by etl_datetime desc limit 1
    """
    print(sql_last_etl)
    df_item=pd.read_sql_query(sql_last_etl, conn)
    print(df_item)
    return df_item
    
   except Exception as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
       logErrorMessage(listError)

if isLoadingAllItems==False:
    dfLastETL=get_last_etl_by_ds(source_name)
    if dfLastETL.empty==False:
      start_date_query=dfLastETL.iloc[0,0]
      print(f"Start Import on update_at of last ETL date :  {start_date_query}" ) 
    else:
       isLoadingAllItems=True 
       start_date_query=init_date_query
       print(f"No etl transaction , we will get started with importing all items from :  {start_date_query}" ) 
else:
   start_date_query=init_date_query
   print(f"Reload all data:  {start_date_query}" ) 
        


Reload all data:  2020-01-01 00:00:00


# Load data from Oracel  as DataFrame 

In [525]:
def loadData(isReLoadAll):
    print(f"isReLoadAll=={isReLoadAll}")  
    try:
       engine = sqlalchemy.create_engine(f"oracle+cx_oracle://{_username}:{_password}@{_ip}:{_port}/?service_name={_servicename}")
       if isReLoadAll==True:
          
         sql=f"""select * from {source_name}   
           where  {colFirstLoad}>=to_date('{start_date_query}','yyyy-mm-dd hh24:mi:ss') 
           """    
       else:    
           sql =f"""select * from {source_name}  
           where  {updateCol}>=to_date('{start_date_query}','yyyy-mm-dd hh24:mi:ss') """
            
       print(sql)
       
       dfAll = pd.read_sql(sql, engine)
    
       return dfAll 
    except Exception as e:
       listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
       logErrorMessage(listError)
    

dfAll=loadData(isLoadingAllItems)


isReLoadAll==True
select * from yip_gl_account   
           where  start_date>=to_date('2020-01-01 00:00:00','yyyy-mm-dd hh24:mi:ss') 
           


# Test Data

In [526]:
# dfAll['last_update_date'].unique()

In [527]:
# dfAll=dfAll.drop(columns=['line_description8','line_description9'  ,'dept_code'])
# _dateColTest='last_update_date'
# _dateValueTest='2023-06-09 22:45:09'
# dfAll=dfAll.query(f"{_dateColTest}<@_dateValueTest")
# dt_imported=datetime.strptime(_dateValueTest,'%Y-%m-%d %H:%M:%S')
# dtStr_imported=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
#=============================================================================


# Transform Dataframe prior to Ingesting it to BQ

In [528]:
dfAll['ImportedAt']=dt_imported 

listColDF=dfAll.columns.tolist()
print("List columns of DF")
print(listColDF)

print(dfAll.info())
dfAll.head()

List columns of DF
['ledger_id', 'ledger_name', 'code_combination_id', 'account_type', 'period_year', 'period_num', 'period_name', 'start_date', 'end_date', 'segment1', 'segment2', 'segment3', 'segment4', 'segment5', 'account_code', 'description', 'period_net_dr', 'period_net_cr', 'period_net', 'begining_balance', 'ending_balance', 'creation_date', 'created_by', 'last_update_date', 'last_updated_by', 'ImportedAt']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321825 entries, 0 to 321824
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   ledger_id            321825 non-null  int64         
 1   ledger_name          321825 non-null  object        
 2   code_combination_id  321825 non-null  int64         
 3   account_type         321825 non-null  object        
 4   period_year          321825 non-null  int64         
 5   period_num           321825 non-null  int64         
 6   pe

,ledger_id,ledger_name,code_combination_id,account_type,period_year,period_num,period_name,start_date,end_date,segment1,...,period_net_dr,period_net_cr,period_net,begining_balance,ending_balance,creation_date,created_by,last_update_date,last_updated_by,ImportedAt
0,61,TCCL Set of Book,59607,E,2023,8,MAY-23,2023-05-01,2023-05-31,12,...,287651.00,472121.52,-184470.52,-287651.00,-472121.52,2023-06-09 10:58:33,2443,2023-06-09 22:16:01,2443,2023-06-13 16:19:18
1,61,TCCL Set of Book,1285170,L,2023,8,MAY-23,2023-05-01,2023-05-31,12,...,976684.80,498635.89,478048.91,-976684.80,-498635.89,2023-06-09 10:58:33,2443,2023-06-09 22:16:01,2443,2023-06-13 16:19:18
2,61,TCCL Set of Book,1536215,E,2023,9,JUN-23,2023-06-01,2023-06-30,12,...,0.00,0.00,0.00,970757.41,970757.41,2023-06-09 10:58:33,2443,2023-06-09 22:16:01,2443,2023-06-13 16:19:18
3,61,TCCL Set of Book,1536215,E,2023,8,MAY-23,2023-05-01,2023-05-31,12,...,970757.41,1264335.80,-293578.39,1264335.80,970757.41,2023-06-09 10:58:33,2443,2023-06-09 22:16:01,2443,2023-06-13 16:19:18
4,61,TCCL Set of Book,59607,E,2023,9,JUN-23,2023-06-01,2023-06-30,12,...,0.00,0.00,0.00,-472121.52,-472121.52,2023-06-09 10:58:33,2443,2023-06-09 22:16:01,2443,2023-06-13 16:19:18


In [529]:
listColAdminConfig=[colFirstLoad,partitionCol]
if len(clusterCols)>0:
 listColAdminConfig.extend(clusterCols)
if len(dateCols)>0:
 listColAdminConfig.extend(dateCols)
listColAdminConfig=list(dict.fromkeys(listColAdminConfig))

checkSomeNotExistingDF = [ col   for col in listColAdminConfig if col not in listColDF ]
if len(checkSomeNotExistingDF)>0:
    msg=f"Some columns in data source are not in dataframe from Oracel View = {checkSomeNotExistingDF }"
    listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg])
    logErrorMessage(listError)
else:
    print(f"{listColAdminConfig} is in dataframe from Oracel View")

['start_date', 'end_date'] is in dataframe from Oracel View


In [530]:
if dfAll.empty:
    print("No row to import to BQ")
    exit()

# BigQuery

## Creaste bigquery schema from dataframe

In [531]:
# schema = [
# bigquery.SchemaField("CUSTOMER_TRX_ID", "INTEGER", mode="NULLABLE"),
# bigquery.SchemaField("GL_DATE", "DATE", mode="NULLABLE"),
# bigquery.SchemaField("DEPT_NAME", "STRING", mode="NULLABLE"),      
# bigquery.SchemaField("INVOICE_AMOUNT", "FLOAT", mode="NULLABLE"),    
# bigquery.SchemaField("LAST_UPDATE_DATE", "TIMESTAMP", mode="NULLABLE"),
# ]
#https://cloud.google.com/bigquery/docs/schemas
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html
schema = []
srCols=dfAll.dtypes
try:
    for name, type_name in srCols.items():
        # print(name,type_name)
        if str(type_name) in ['int32','int64']:
          schema.append(bigquery.SchemaField(name, "INTEGER", mode="NULLABLE"))
        elif str(type_name) =='float64':
          schema.append(bigquery.SchemaField(name, "FLOAT", mode="NULLABLE"))
        elif str(type_name) =='datetime64[ns]':
          if name in   dateCols:
             schema.append(bigquery.SchemaField(name,  "DATE", mode="NULLABLE"))
          else:
             schema.append(bigquery.SchemaField(name,  "DATETIME", mode="NULLABLE"))
        elif str(type_name) == 'bool':
             schema.append(bigquery.SchemaField(name,  "BOOL", mode="NULLABLE"))
        else: # if not found type , it will be converted to STRING
           schema.append(bigquery.SchemaField(name,  "STRING", mode="NULLABLE")) 
        
    # if  len(schema)!=len(listColDF):
    #    listFieldBQError=[field.name for field in schema]
    #    intersec_DF_BQ = [set(listColDF).symmetric_difference(set(listFieldBQError))]
    #    raise Exception(f"{len(schema)}!={len(listColDF)} , {intersec_DF_BQ} in Dataframe can not be converted to Bigquery Data type")

except Exception as e:
   listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
   logErrorMessage(listError)
print("Total number of field of the schema :",len(schema))    
print(schema) 

Total number of field of the schema : 26
[SchemaField('ledger_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('ledger_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('code_combination_id', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('account_type', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('period_year', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('period_num', 'INTEGER', 'NULLABLE', None, None, (), None), SchemaField('period_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('start_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('end_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('segment1', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('segment2', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('segment3', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('segment4', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('segment5', 'STRING'

## Check whether dataframe and bigquery schema are the same

## Check Existing  Table

In [532]:
def create_table():
    try:  
        table = bigquery.Table(table_id,schema=schema)
        if  partitionCol!="":
         table.time_partitioning = bigquery.TimePartitioning(
         type_=partitionType,field=partitionCol)

        if len(clusterCols)>0:
         table.clustering_fields = clusterCols

        table = client.create_table(table) 
        print(
            "Created new table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
        )
    except Exception as e:
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)]) 
        logErrorMessage(listError)   

In [533]:
def get_table():
    try:
        client.get_table(table_id)
        return True
    except NotFound:
        return False

In [534]:
# def check_same_schema(listFieldBQ,partitionNameBQ,partitionTypeBQ,clusterBQ,dateTypeBQ):
def check_same_schema():
    print("===============================================================================================")
    print("Check every columns name and partition cluster and date type column on table against dataframe")
    def find_difference(dfX,bqX):
        intersec_DF_BQ = [set(dfX).symmetric_difference(set(bqX))]
        list_DF_BQ=[]
        if len(intersec_DF_BQ)>0:
         for item in intersec_DF_BQ:
            list_DF_BQ=list_DF_BQ+list(item)
        return list_DF_BQ 
        
    listColumnX=find_difference(listColDF,listFieldBQ)
    if len(listColumnX)>0:
        e=f"Columns: {listColumnX} are NOT THE SAME on BigQuery and ViewTable {source_name} of Oracle"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("All Fields on BQ and DF are ok.")
        
        
    # PartitionName
    if partitionNameBQ!=partitionCol:
        e=f"Partition Column :{partitionNameBQ} in BQ is NOT THE SAME as {partitionCol} defined on ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("Partition Name Fields on BQ and DF is ok.")    
        

    # # PartitionDateType
    if partitionTypeBQ!=partitionType:
        e=f"Partition Date Type :{partitionTypeBQ} in BQ is NOT THE SAME as {partitionType} defined on ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    
    # Cluster List
    listClusterX=find_difference(clusterCols,clusterBQ)
    if len( listClusterX)>0:
        e=f"Cluster columns : {listClusterX} are NOT THE SAME on BigQuery and ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("All Cluster on BQ and DF are ok.")
    
    # Date Type List
    
        # Cluster List
    listDateColX=find_difference(dateCols,dateTypeBQ)
    if len( listDateColX)>0:
        e=f"Date columns : {listDateColX} are NOT THE SAME on BigQuery and ETL Config on Web Admin"
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    else:
        print("All Date Column on BQ and DF are ok.")

    if len(listError)>0:
        logErrorMessage(listError)
        
# delete table and set isLoading=True to load all data
        
#       isLoadingAllItems=True
#       start_date_query=init_date_query

#       print("ReLoad Data due to something in schema changed")  
#       dfAll=loadData(isLoadingAllItems)
#       print(dfAll.info())  

#       print("Delete table and re-create new one.")
#       client.delete_table(table_id, not_found_ok=True)  
#       create_table()
 
  

In [535]:
isExistingTable=get_table()
if isExistingTable:    
    table=client.get_table(table_id)
    print("Table {} already exists.".format(table_id))
    
    listFieldBQ=[field.name for field in table.schema]
    
    # required field
    partitionNameBQ=table.time_partitioning.field
    partitionTypeBQ=table.partitioning_type

    clusterBQ=table.clustering_fields
    if clusterBQ is None : clusterBQ =[]
        
    dateTypeBQ=[field.name for field in table.schema if field.field_type=='DATE']
    
    
    print(f"All Fields : {listFieldBQ}")
    print(f"Partiton Field&Type: {partitionNameBQ} - {partitionTypeBQ}")
    print(f"Cluster Field List: {clusterBQ}")
    print(f"Date Field List: {dateTypeBQ}")
    
    check_same_schema()
else:
    create_table()
        


Created new table mismgntdata-bigquery.MIS_BI_DW.yip_gl_account


# Load data from CSV file to BQ

In [536]:
no_rows=len(dfAll)
no_cols=len(dfAll.columns)
if dfAll.empty==False:
 if loading_from=='csv':
    dfAll.to_csv (temp_path,index=False)
print(f"{no_rows} rows and {no_cols} are about to be imported to BQ by {loading_from}")

321825 rows and 26 are about to be imported to BQ by dataframe


In [537]:
# cannot auto detect because some column , there are Y,N,R  For R BQ is known as Bool
#https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

def collectBQError(x_job):
 if x_job.errors is not None:
    for error in x_job.errors:  
      msg=f"{error['reason']} - {error['message']}"
      listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,msg])
    if   len(listError)>0:
     logErrorMessage(listError,False)

try:
    print(f"Load bulk data from {loading_from}")
    if loading_from=='csv' :
        
        job_config = bigquery.LoadJobConfig(
            source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1,
            schema=schema,autodetect=False,
            max_bad_records=(no_rows-1),
            # autodetect=True,
            write_disposition="WRITE_APPEND",
            )
        with open(temp_path, "rb") as source_file:
            job = client.load_table_from_file(source_file, table_id, job_config=job_config)
    else :
        # job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")  # ok for POs Listing
        
        job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND",schema=schema)
        job = client.load_table_from_dataframe(dfAll, table_id, job_config=job_config,)  


    result_job=job.result()  # Waits for the job to complete.
    
    # error but continue
    collectBQError(job)   
    
    print(f"Import data from {source_name} on Oracle into BQ successfully.")
   


except Exception as e:
  print("BQ Error and Raise Exception")  
  print(str(e))
  collectBQError(job)
    

Load bulk data from dataframe
Import data from yip_gl_account on Oracle into BQ successfully.


# Create Transation and delete csv file

In [538]:

#Addtional Try Error    
def insertETLTrans(recordList):
    try:
        sqliteConnection = sqlite3.connect(os.path.abspath(data_base_file))
        cursor = sqliteConnection.cursor()
        sqlite_insert_query = """
        INSERT INTO etl_transaction
        (etl_datetime, data_source_id,is_load_all,completely)  VALUES (?,?,?,?);
         """

        cursor.executemany(sqlite_insert_query, recordList)
        print("Done ETL Trasaction")
        sqliteConnection.commit()
        cursor.close()

    except Exception as e:
        listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
        logErrorMessage(listError)
        print("Failed to insert etl_transaction table", str(e))
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            


if isLoadingAllItems==True:
    is_load_all=1
else:
    is_load_all=0
    
if len(listError)>0:
 is_loaded_completely=0
else:
 is_loaded_completely=1

#dfETFTran=pd.DataFrame.from_records([{'etl_datetime':dtStr_imported,'data_source_id':source_name,'no_rows':no_rows,'is_load_all':is_load_all}])
dfETFTran=pd.DataFrame.from_records([{'etl_datetime':dtStr_imported,'data_source_id':source_name,'is_load_all':is_load_all,'completely':is_loaded_completely}])
recordsToInsert=list(dfETFTran.to_records(index=False))
insertETLTrans(recordsToInsert)

Done ETL Trasaction


In [539]:
#Addtional Try Error
try:
    if os.path.exists(temp_path):
      os.remove(temp_path)
      print(f"Deleted {temp_path}")
    
except Exception as e:
    listError.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S"),dtStr_imported,source_name,str(e)])
    logErrorMessage(listError)